In [20]:
import pandas as pd
import time
import numpy as np
from scipy import sparse
from scipy.stats import skew, boxcox
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
import gc

In [16]:
def log_mae(labels,preds,lift=200):
    return mean_absolute_error(np.exp(labels)-lift, np.exp(preds)-lift)

# custom metric function for Keras
def mae_log(y_true, y_pred): 
    return K.mean(K.abs((K.exp(y_pred)-200) - (K.exp(y_true)-200)))


# Keras deosn't support sparse matrix. 
# The following functions are useful to split a large sparse matrix into smaller batches so they can be loaded into mem.
def batch_generator(X, y, batch_size, shuffle):
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0
            
def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0            

## Load Data

In [3]:
# Load data
start = time.time() 
train_data = pd.read_csv('../input/train.csv')
train_size=train_data.shape[0]
print ("Loading train data finished in %0.3fs" % (time.time() - start))        

test_data = pd.read_csv('../input/test.csv')
print ("Loading test data finished in %0.3fs" % (time.time() - start))         

Loading train data finished in 2.915s
Loading test data finished in 4.693s


#### Merge train and test

This will save our time on duplicating logics for train and test and will also ensure the transformations applied on train and test are the same.

In [4]:
full_data=pd.concat([train_data,test_data])
del( train_data, test_data)
print ("Full Data set created.")

Full Data set created.


In [5]:
data_types = full_data.dtypes  
cat_cols = list(data_types[data_types=='object'].index)
num_cols = list(data_types[data_types=='int64'].index) + list(data_types[data_types=='float64'].index)

id_col = 'id'
target_col = 'loss'
num_cols.remove('id')
num_cols.remove('loss')

### Numeric features

Two preprocessings on numeric features are applied:

   1. Apply box-cox transformations for skewed numeric features.

   2. Scale numeric features so they will fall in the range between 0 and 1.



In [6]:
skewed_cols = full_data[num_cols].apply(lambda x: skew(x.dropna()))

SSL = preprocessing.StandardScaler()
skewed_cols = skewed_cols[skewed_cols > 0.25].index.values
for skewed_col in skewed_cols:
    full_data[skewed_col], lam = boxcox(full_data[skewed_col] + 1)
for num_col in num_cols:
    full_data[num_col] = SSL.fit_transform(full_data[num_col].values.reshape(-1,1))

# Model OHE Coding
####  Categorical features
1. Label Encoding (Factorizing)
2. One Hot Encoding (get dummies)

OHE can be done by either get_dummies() from Pandas package or OneHotEncoder from SK-Learn package. 

* get_dummies is easier to implement (can be used directly on raw categorical features, i.e. strings, but it takes longer time and is not memory efficient.

* OneHotEncoder requires the features being converted to numeric, which has already been done by LabelEncoder in previous step, and is much more efficient (7x faster).

* The OHE's results are converted to a sparse matrix which uses way less memory as compared to dense matrix.

In [7]:
LBL = preprocessing.LabelEncoder()
start=time.time()
for cat_col in cat_cols:
    full_data[cat_col] = LBL.fit_transform(full_data[cat_col])
print ('Label enconding finished in %f seconds' % (time.time()-start))

Label enconding finished in 33.628522 seconds


In [8]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
full_data_sparse=OHE.fit_transform(full_data[cat_cols])
print 'One-hot-encoding finished in %f seconds' % (time.time()-start)
print full_data_sparse.shape

## it should be (313864, 1176)

One-hot-encoding finished in 10.209203 seconds
(313864, 1176)


In [9]:
lift = 200

full_data_sparse = sparse.hstack((full_data_sparse
                                  ,full_data[num_cols])
                                 , format='csr'
                                 )
print full_data_sparse.shape
train_x = full_data_sparse[:train_size]
test_x = full_data_sparse[train_size:]
train_y = np.log(full_data[:train_size].loss.values + lift)
ID = full_data.id[:train_size].values

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, train_size=.80, random_state=1234)

(313864, 1190)


### [Kearas](https://keras.io)

Keras is a high-level neural networks library, written in Python and capable of running on top of either TensorFlow or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU,LeakyReLU,ELU,ParametricSoftplus,ThresholdedReLU,SReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import SGD,Nadam
from keras.regularizers import WeightRegularizer, ActivityRegularizer,l2, activity_l2

Using Theano backend.


#### Keras starter

Below is a quick starter example for creating a neural networks model using Keras. It covers the following aspects:
1. multiple layers: 1 input, 1 hidden and 1 output
2. normalization.
3. dropout regularization.
4. early stopping
5. activate function
6. optimizer
7. batch training

For manual tunning of the neural networks model, I follow the instruction from [Tilii](https://www.kaggle.com/tilii7). The [link1](https://www.kaggle.com/mtinti/allstate-claims-severity/keras-starter-with-bagging-1111-84364/comments#143686) and [link2](https://www.kaggle.com/mtinti/allstate-claims-severity/keras-starter-with-bagging-1111-84364/comments#143693) are the original post from him.

    Step 1: tune the size of each layer
    Step 2: tune optimizer
    Step 3: tune initialization function
    Step 4: tune activation function
    Step 5: tune dropout


In [21]:
early_stop = EarlyStopping(monitor='val_mae_log', patience=5, verbose=0, mode='auto')
checkpointer = ModelCheckpoint(filepath="weights.hdf5", monitor='val_mae_log', verbose=1, save_best_only=True, mode='min')

def nn_model(params):
    model = Sequential()
    
    # input layer
    model.add(Dense(params['input_size'], input_dim = params['input_dim']))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(params['input_drop_out']))
    
    # 1st hidden layer
    model.add(Dense(params['hidden_size0']))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(params['hidden_drop_out0']))
    
    # 2nd hidden layer
    model.add(Dense(params['hidden_size1']))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(params['hidden_drop_out1']))    
    
    # output layer
    model.add(Dense(1))
    
    model.compile(loss = 'mae', metrics=[mae_log], optimizer = 'adadelta')
    return(model)


def nn_blend_data(parameters, train_x, train_y, test_x, fold, early_stopping_rounds=0, batch_size=128):
    print ("Blend %d estimators for %d folds" % (len(parameters), fold))
    skf = list(KFold(len(train_y), fold))
    
    scores = np.zeros ((len(skf),len(parameters)))
    best_rounds = np.zeros ((len(skf),len(parameters)))
 
    for j, nn_params in enumerate(parameters):
        print ("Model %d: %s" %(j+1, nn_params))
        test_blend_x_j = np.zeros((test_x.shape[0], len(skf)))
        for i, (train, val) in enumerate(skf):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train]
            train_y_fold = train_y[train]
            val_x_fold = train_x[val]
            val_y_fold = train_y[val]

            model = nn_model(nn_params)
            print (model)
            fit= model.fit_generator(generator=batch_generator(train_x_fold, train_y_fold, batch_size, True),
                                     nb_epoch=60,
                                     samples_per_epoch=train_x_fold.shape[0],
                                     validation_data=(val_x_fold.todense(), val_y_fold),
                                     verbose = 1,
                                     callbacks=[
#                                                 EarlyStopping(monitor='val_mae_log'
#                                                               , patience=early_stopping_rounds, verbose=0, mode='auto'),
                                                ModelCheckpoint(filepath="weights.hdf5"
                                                                , monitor='val_mae_log', 
                                                                verbose=0, save_best_only=True, mode='min')
                                                ]
                                     )
            
            model.load_weights("weights.hdf5")
            # Compile model (required to make predictions)
            model.compile(loss = 'mae', metrics=[mae_log], optimizer = 'adadelta')

            val_y_predict_fold = model.predict_generator(generator=batch_generatorp(val_x_fold, batch_size, True),
                                        val_samples=val_x_fold.shape[0]
                                     )
            
            score = log_mae(val_y_fold, val_y_predict_fold,200)
            print ("Score: ", score, mean_absolute_error(val_y_fold, val_y_predict_fold))
            scores[i,j]=score

            # Compile model (required to make predictions)
            model.compile(loss = 'mae', metrics=[mae_log], optimizer = 'adadelta')            
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
   
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    return (scores)

In [ ]:
nn_parameters = [
    {'input_size' :400 ,
     'input_dim' : train_x.shape[1],
     'input_drop_out' : 0.5 ,
     'hidden_size0' : 150 ,
     'hidden_drop_out0' :0.1,
     'hidden_size1' : 20 ,
     'hidden_drop_out1' :0.05,    
     'learning_rate': 0.1}
]

(train_blend_x, test_blend_x, blend_scores,best_round) = nn_blend_data(nn_parameters, train_x, train_y, test_x,
                                                         4,
                                                         5)